Para responder puedes crear las celdas que quieras, crear codigo,  incluir imagenes adicionales, platicarlo con tus companneres, preguntar, usar ChatGPT o lo que sea. Solo asegurate de hacerlo explicable, entender lo que esta pasando y ser capaz de defender tus elecciones.

Escribe tu nombre y clave

Nombre: Diego Arellano Zamudio
Clave: 198002

Los cambios se hicieron a la función model_selection. Específicamente, se agregó el parámetro class_weight con el valor 'balanced' a los modelos que lo aceptan (Logistic Regression y Random Forest Classifier). Esto logra ponderar la función de costos con base en las proporciones de las clases dentro del conjunto de datos. 

In [ ]:
def model_selection(df_train, num_folds=3, seed = 7364, scoring=["accuracy","balanced_accuracy","roc_auc", "f1"]):
    X_train = df_train.drop(columns=['y'])
    Y_train = df_train['y']

    mm_scaler = preprocessing.MinMaxScaler()
    X_train = mm_scaler.fit_transform(X_train)

    # spot check the algorithms
    models = []
    #models.append(('DC', DummyClassifier(strategy="most_frequent")))
    models.append(('LR', LogisticRegression(max_iter=500, class_weight='balanced'))) # Este código añadí
    models.append(('LDA', LinearDiscriminantAnalysis()))
    models.append(('KNN', KNeighborsClassifier()))
    #models.append(('CART', DecisionTreeClassifier()))
    #models.append(('NB', GaussianNB()))
    #Neural Network
    models.append(('NN', MLPClassifier(max_iter=500, solver='sgd', 
                                    early_stopping=True)))
    #Ensable Models 
    # Boosting methods
    #models.append(('AB', AdaBoostClassifier()))
    models.append(('GBM', GradientBoostingClassifier()))
    # Bagging methods
    models.append(('RF', RandomForestClassifier(class_weight='balanced'))) # Este código añadí
    #models.append(('ET', ExtraTreesClassifier()))

    # ##Opciones

    results = [] 

    for name, model in models:
        print(f"Model: {name}")
        kfold = KFold(n_splits=num_folds, random_state=seed, shuffle=True)
        cv_results = cross_validate(model, X_train, Y_train, cv=kfold,
                                    scoring=scoring, n_jobs=-1)
        cv_results = pd.DataFrame(cv_results)
        #print(cv_results.mean())
        # msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
        # print(msg)
        # cv_results = pd.DataFrame({
        #     'results':cv_results
        # })
        cv_results.loc[:,'model_name'] = name
        results.append(cv_results)

    # Convertir todos los resultados un solo DataFrame
    results = pd.concat(results, ignore_index=True)
    print(results.groupby(['model_name'], as_index=False).describe())
    results_plot =  [c for c in results.columns if 'time' not in c]
    results_plot = results.loc[:,results_plot]
    results_plot = pd.melt(
        results_plot, id_vars='model_name', var_name="metric")
    #results_plot = results_plot[results_plot['metric']]
    g = sns.FacetGrid(results_plot, row="metric",
                      height=3, aspect=1, sharex=False)
    g.map(sns.boxplot, 'model_name','value')
    g.fig.set_size_inches(13, 20)
    
    return results

Explica por qué elegiste los pesos que usaste (o el método), y si estos deben de ser incluidos en el test también.

El método para calcular el peso para cada clase es:

peso_clase_i = Número total de muestras / (Número de clases * Número de muestras de la clase i)

que modifica a la función de pérdida de tal manera que las clases más raras tengan mayor peso. 

Estos pesos no deben ser incluidos en el test, sólo se usan para entrenar al modelo. 




# Responde las siguientes preguntas sobre los modelos (sklearn)

1. Que modelo(s) no estan sobreajustando (overfitting) sin importar la metrica? 


Respuesta
Los modelos que no están sobreajustando sin importar si las clases están o no balanceadeas son: Linear Discriminant Analysis, K Neighbors y el Gradient Boosting. 

2. Que modelos tienen mayor riesgo de sobreajustar (overfitting) dependiendo de la metrica? Y por que?

Respuesta:
El modelo que tiene mayor riesgo de sobreajustar es el modelo de regresión logística, ya que toma en cuenta la probabilidad de las etiquetas y puede conducir a un sesgo a la clase mayoritaria. 
También los árboles de decisión tienen riesgo si no se controla adecuadamente el hiperparámetro de la profundidad, un árbol muy profundo hace que aprenda muy bien los datos de entrenamiento. 

# Responde las siguientes preguntas generales (sklearn)

1. Las metricas (todas, algunas?) de sub, over, ponderadas y sin sampleo son comparables entre si o no? (Cuales son comparables entre si y bajo que criterios?)

Respuesta:

Las métricas en general no son comparables, ya que al hacer subsampleo u oversampleo cambiamos la distribución de los datos, lo que puede afectar las tasas de verdaderos positivos, verdaderos negativos, falsos positivos y falsos negativos. Sin embargo la métrica de balanced accuracy sí se puede usar para comparar los modelos independietemente de la técnica de muestreo, ya que es la media entre la sensibilidad y la especificidad. También se pueden comparar con la métrica F1 y con AUC. 

2. Debes aplicar sub, over sampleo y poderacion al conjunto de test para poder comparar las metricas que se crearon durante el entrenamiento con las de test?

Respuesta:
Al conjunto de prueba no se le debe de aplicar la técnica de sampleo ya que estos datos deben de reflejar los datos que se encuentran en la realidad. 

3. Que metricas (incluyendo la matriz confusion) te parecen las mejores para elegir resultados y decidir que modelo entrara a produccion? (y por que?)

Respuesta: 
Las métricas que me parecen mejores son la balanced accuracy, F1 y AUC ya que toman en cuenta el desbalanceo de los datos y se pueden comparar dependiendo de la técnica de sampleo que se utilizó. También me parece que la matriz de confusión es buena métrica para decidir el modelo si tenemos un objetivo en específico (por ejemplo maximizar el número de verdaderos positivos o minimizar los falsos positivos).

## Responde de acuerdo a tu criterio

1. Si yo fuera el banco y te mencionara que actualmente no tenemos un benchmark para que puedas comparar tu modelo cual seria tu propuesta de bechmark (modelo y metricas)?

Respuesta: 
Teniendo en cuenta que un buen modelo benchmark debe ser simple y de rendimiento razonable yo propondría el modelo de regresión logística. La regresión logística es un modelo fácilmente interpretable, tuvo buen rendimiento y es fácil de entrenar. Respecto a la métrica, yo elegiría la AUC ya que igual es una métrica fácil de interpretar y nos ayuda a evaluar aún teniendo un dataset desequilibrado. 

2.  Que metodo utilizarias para produccion? Explica tus razones de manera extensiva, incluye la eleccion de algoritmo, metrica, tipo de sampleo, analisis de varianza y bias de resultados y utilidad para el banco. Osea basicamente convenceme de manera tecnica pq elegirias todo eso en la realidad.

Respuesta: 

Yo elegiría el modelo de Gradient Boosting Classifier porque es el modelo que tiene el mejor desempeño considerando todas las formas de sampleo. Para la métrica consideraría la AUC ya que mide la capacidad del modelo para distinguir entre la clase positiva y negativa, además de que es robusta ante datasets desequilibrados. Esto sería útil para el banco ya que podría minimizar el número de default sin exagerar en el número de casos que no presta. Respecto al tipo de sampleo, no usaría oversampling ni undersampling, ya que el desempeño en AUC es mejor sin usar estas técnicas. Por otro lado, analizando la matriz de confusión, con over y undersampling sí detectamos más verdaderos positivos, pero a cambio de aumentar la tasa de falsos positivos considerablemente y por lo tanto no prestándole a más gente que sí iba a pagar. Por último, el modelo está ajustado correctamente, ya que su rendimiento en el test es bueno. Además, no tiene niveles altos de sesgo ya que sí predice la clase de interés lo suficiente como para decir que aprendió de los datos de entrenamiento y no tiene alta varianza ya que el error no fluctúa demasiado. 